In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# default_exp perturb

# Perturb

> Functions for perturbing a graph 

## Edge deletions

In [ ]:
#export 
from nbdev.showdoc import *
import networkx as nx
import numpy as np
from grapht.graphtools import non_pendant_edges, has_isolated_nodes
from grapht.sampling import khop_subgraph, sample_edges

In [ ]:
#export 
def khop_edge_deletion(G, k, r):
    """
    Removes r edges which are in a k-hop neighbourhood of some node, the perturbed graph will not have isolated nodes
    
    If k is None then the samples are taken uniformly
    """
    solution = None
    while solution is None:
        subgraph, node = khop_subgraph(G, k) if k is not None else (G, None)
        if len(non_pendant_edges(subgraph)) < r:
            continue
        edges = sample_edges(subgraph, r, non_pendant=True)
        Gp = G.copy()
        Gp.remove_edges_from(edges)
        if not has_isolated_nodes(Gp):
            solution = Gp
    return solution, edges, node

## Rewiring

In [ ]:
#export
def khop_rewire(G, k, r, max_iter=np.Inf):
    """Rewiring in a k-hop neighbourhood"""
    solution, iteration = None, 0
    while solution is None:
        iteration = iteration + 1
        if iteration == max_iter:
            return None
        subgraph, node = khop_subgraph(G, k) if k is not None else (G, None)
        if len(subgraph.edges()) < r:
            continue
        edges = sample_edges(subgraph, r, non_pendant=False)
        Gp = G.copy()
        rewire_info = rewire(Gp, edges)
        if not has_isolated_nodes(Gp):
            solution = Gp
    return solution, rewire_info, node 
        
        
def rewire(G, edges):
    """
    Rewires edges in G. Each row in the returned array is (u ,v ,newv) where (u, v) was removed and (u, newv) was added
    """
    G.remove_edges_from(edges)
    edges = np.array(edges)
    new_endpoints = np.random.permutation(edges[:, 1])
    edges = np.concatenate([edges, np.expand_dims(new_endpoints, 1)], axis=1)
    G.add_edges_from(edges[:,[0, 2]].tolist())
    G.remove_edges_from(nx.selfloop_edges(G))
    return edges

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_graphtools.ipynb.
Converted 01_sampling.ipynb.
Converted 02_metrics.ipynb.
Converted 03_perturb.ipynb.
Converted 04_plotting.ipynb.
Converted 05_data.ipynb.
Converted index.ipynb.
